In [ ]:
import torch
import torch.nn as nn
import re
from torch.nn import functional as F

#hyperparams
embedding_dim = 8
batch_size = 8 # Number of sequences to process in parallel
block_size = 32 # Maximum context length for predictions
num_heads = 8
num_layers = 6
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-3
max_iters = 5000
eval_interval = 100
eval_iters = 200
#-------

torch.manual_seed(1337)

with open('/content/drive/MyDrive/churchill.txt', 'r', encoding='utf-8') as f:
    text = f.read()


def split_and_keep_spaces(text):
    # Split the text by spaces while keeping the spaces in the list
    result = re.split(r'(\s)', text)
    return result

words = split_and_keep_spaces(text=text)
tokens = sorted(list(set(words)))
num_unique_tokens = len(tokens)


#word_embedding = nn.Embedding(num_embeddings=num_unique_tokens, embedding_dim=embedding_dim)

#Create a mapping from tokens to ints:
tok_to_int = { token:i for i,token in enumerate(tokens)}
int_to_token = { i:token for i,token in enumerate(tokens)}
encode = lambda seq: [tok_to_int[t] for t in seq] #Encoder: Sequence to list of ints
decode = lambda l: ''.join([int_to_token[i] for i in l]) #Decoder: list of ints to Sequence

#Train and test splits
data = torch.tensor(encode(words), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

#Dataloader
def get_batch(split):
    #Generate batch of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Transformer(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(num_embeddings=num_unique_tokens, embedding_dim=embedding_dim)
        self.position_embedding_table = nn.Embedding(num_embeddings=block_size, embedding_dim=embedding_dim)
        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=embedding_dim)
        self.encoder = nn.TransformerEncoder(encoder_layer=self.encoder_layer, num_layers=num_layers, norm=self.layer_norm)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=embedding_dim)
        self.decoder = nn.TransformerDecoder(decoder_layer=self.decoder_layer, num_layers=num_layers, norm=self.layer_norm)
        self.feedforward = nn.Linear(embedding_dim, num_unique_tokens)
        self.softmax = F.softmax

    def forward(self, idx, targets=None):
        # (batch, sequence_length, embedding_dim)
        B, S, E = 0, 0, 0
        B, S = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,S,E)
        pos_emb = self.position_embedding_table(torch.arange(S, device=device)) # (T, E)

        if targets != None:
            tgt_emb = self.token_embedding_table(targets)
        else:
            tgt_emb = tok_emb

        tgt = tgt_emb + pos_emb

        x = tok_emb + pos_emb # (B,S,E)
        memory = self.encoder(x)
        x = self.decoder(tgt, memory)

        logits = self.feedforward(x)

        if targets is None:
            loss = None
        else:
            B, S, E = logits.shape
            logits = logits.view(B*S, E)
            targets = targets.view(B*S)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        #idx is (B, S) array
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = Transformer()
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


context = split_and_keep_spaces("bearing the brunt of German malice and cruelty.")
empty = torch.tensor(encode(context), dtype=torch.long, device=device)
empty.shape
context = empty.view((1, empty.__len__()))
#print(context)
print(decode(m.generate(context, max_new_tokens=20)[0].tolist()))

step 0: train loss 9.6573, val loss 9.6517
step 100: train loss 7.6881, val loss 7.7020
step 200: train loss 5.8426, val loss 5.8734
step 300: train loss 4.3263, val loss 4.3879
step 400: train loss 3.7759, val loss 3.8316
step 500: train loss 3.5301, val loss 3.5887
step 600: train loss 3.3619, val loss 3.4294
step 700: train loss 3.2207, val loss 3.2839
step 800: train loss 3.1005, val loss 3.1774
step 900: train loss 2.9864, val loss 3.0589
step 1000: train loss 2.8833, val loss 3.0019
step 1100: train loss 2.8020, val loss 2.9059
step 1200: train loss 2.7247, val loss 2.8353
step 1300: train loss 2.6248, val loss 2.7602
step 1400: train loss 2.5488, val loss 2.6595
step 1500: train loss 2.4912, val loss 2.6048
step 1600: train loss 2.4138, val loss 2.5430
step 1700: train loss 2.3430, val loss 2.4815
step 1800: train loss 2.2761, val loss 2.4337
step 1900: train loss 2.2219, val loss 2.3731
step 2000: train loss 2.1808, val loss 2.3014
step 2100: train loss 2.1250, val loss 2.2833
